In [2]:
%load_ext rpy2.ipython

In [3]:
%%R
R.version.string

[1] "R version 4.5.1 (2025-06-13)"


In [ ]:
%%R
library(rfishbase)
library(readr)
library(dplyr)

In [ ]:
%%R
ncbi_aphia_species <- read.csv("new_aphia_ncbi_all_18_09_2025.csv")

head(ncbi_aphia_species)
nrow(ncbi_aphia_species)

[1] 19816


In [ ]:
%%R
# Load your species list (Anushka's new_aphia_ncbi_all_18_09_2025 data csv (raw species list in there))
ncbi_aphia_species <- read_csv("C:/Users/30931/OneDrive/Desktop/ocean genome/all fishbase/new_aphia_ncbi_all_18_09_2025.csv")

head(ncbi_aphia_species)
nrow(ncbi_aphia_species)

Rows: 19816 Columns: 26
-- Column specification --------------------------------------------------------
Delimiter: ","
chr  (17): ncbi_taxon_id, species, subspecies, family_x, target_list_status,...
dbl   (8): aphia_id, valid_AphiaID, isMarine, isBrackish, isFreshwater, isTe...
dttm  (1): modified

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.
[1] 19816


In [ ]:
%%R
# fetch fishbase data for all species
fishbase_dat_demo <- species(ncbi_aphia_species$species,
                            fields = c('SpecCode', 'Genus', 'Species', 'Author',
                            'FBname', 'DemersPelag', 'Subfamily', 'DepthRangeShallow',
                            'DepthRangeDeep', 'DepthRangeComShallow', 'DepthRangeComDeep',
                            'Length', 'CommonLength', 'Weight'))

Joining with `by = join_by(SpecCode)`


In [ ]:
%%R
# merge back to our raw species table
sp_joined <- ncbi_aphia_species %>% left_join(fishbase_dat_demo, by = c('species' = 'Species')) %>%
  rename(spec_code = SpecCode,
         depth_min_in_m   = DepthRangeShallow,
         depth_max_in_m   = DepthRangeDeep,
         common_depth_min = DepthRangeComShallow,
         common_depth_max = DepthRangeComDeep,
         length_max_in_cm  = Length,
         common_length_in_cm = CommonLength,
         weight_max_in_g  = Weight)

# inspect
glimpse(sp_joined)

## drop duplicate columns
sp_joined_final <- sp_joined %>%
  select(-genus)

Rows: 19,816
Columns: 39
$ ncbi_taxon_id       <chr> "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", ~
$ species             <chr> "Acanthurus fowleri", "Acanthurus mindorensis", "A~
$ subspecies          <chr> "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", ~
$ family_x            <chr> "Acanthuridae", "Acanthuridae", "Acanthuridae", "C~
$ target_list_status  <chr> "long_list", "long_list", "long_list", "long_list"~
$ sequencing_status   <chr> "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", ~
$ synonym             <chr> "Acanthurus fowleri", "Acanthurus mindorensis", "A~
$ publication_id      <chr> "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", ~
$ aphia_id            <dbl> 272960, 1493122, 272965, 275288, 275289, 275290, 3~
$ url                 <chr> "https://www.marinespecies.org/aphia.php?p=taxdeta~
$ scientificname      <chr> "Acanthurus fowleri", "Acanthurus mindorensis", "A~
$ authority           <chr> "de Beaufort, 1951", "Herre, 1927", "Briggs & Cald~
$ status       

In [ ]:
# pass result back to python
%R -o sp_joined_final

In [ ]:
# clean NA values
import numpy as np

sp_joined_final = sp_joined_final.replace(-2147483648, np.nan)
print(sp_joined_final.head())

  ncbi_taxon_id                 species subspecies      family_x  \
1             -      Acanthurus fowleri          -  Acanthuridae   
2             -  Acanthurus mindorensis          -  Acanthuridae   
3             -     Acanthurus randalli          -  Acanthuridae   
4             -     Antigonia aurorosea          -     Caproidae   
5             -           Antigonia eos          -     Caproidae   

  target_list_status sequencing_status                 synonym publication_id  \
1          long_list                 -      Acanthurus fowleri              -   
2          long_list                 -  Acanthurus mindorensis              -   
3          long_list                 -     Acanthurus randalli              -   
4          long_list                 -     Antigonia aurorosea              -   
5          long_list                 -           Antigonia eos              -   

    aphia_id                                                url  ...  \
1   272960.0  https://www.marine

In [ ]:
# save as csv
sp_joined_final.to_csv("C:/Users/30931/OneDrive/Desktop/ocean genome/all fishbase/sp_joined_fishbase_full.csv", index=False)

OSError: Cannot save file into a non-existent directory: 'C:\Users\30931\OneDrive\Desktop\ocean genome\all fishbase'

## Synonym resolution

In [ ]:
print('Number of rows in sp_joined_final:', len(sp_joined_final))
sp_joined_final.head()

Number of rows in sp_joined_final: 19816


,ncbi_taxon_id,species,subspecies,family_x,target_list_status,sequencing_status,synonym,publication_id,aphia_id,url,...,FBname,DemersPelag,Subfamily,depth_min_in_m,depth_max_in_m,common_depth_min,common_depth_max,length_max_in_cm,common_length_in_cm,weight_max_in_g
1,-,Acanthurus fowleri,-,Acanthuridae,long_list,-,Acanthurus fowleri,-,272960.0,https://www.marinespecies.org/aphia.php?p=taxd...,...,Fowler's surgeonfish,reef-associated,Acanthurinae,2.0,50.0,2.0,20.0,45.0,NaN,NaN
2,-,Acanthurus mindorensis,-,Acanthuridae,long_list,-,Acanthurus mindorensis,-,1493122.0,https://www.marinespecies.org/aphia.php?p=taxd...,...,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-,Acanthurus randalli,-,Acanthuridae,long_list,-,Acanthurus randalli,-,272965.0,https://www.marinespecies.org/aphia.php?p=taxd...,...,Gulf surgeonfish,reef-associated,Acanthurinae,1.0,20.0,NaN,NaN,18.0,NaN,NaN
4,-,Antigonia aurorosea,-,Caproidae,long_list,-,Antigonia aurorosea,-,275288.0,https://www.marinespecies.org/aphia.php?p=taxd...,...,None,bathydemersal,None,280.0,310.0,NaN,NaN,NaN,NaN,NaN
5,-,Antigonia eos,-,Caproidae,long_list,-,Antigonia eos,-,275289.0,https://www.marinespecies.org/aphia.php?p=taxd...,...,None,benthopelagic,None,0.0,311.0,NaN,NaN,12.4,NaN,NaN


### Check missing IDs (both ncbi and aphia ids)

*   List item

*   List item
*   List item


*   List item



In [ ]:
import numpy as np
import pandas as pd
# standardise missing values
for col in ["ncbi_taxon_id", "aphia_id"]:
  sp_joined_final[col] = sp_joined_final[col].replace(["-", ""], np.nan)
  sp_joined_final[col] = pd.to_numeric(sp_joined_final[col], errors="coerce")

In [ ]:
missing_both = sp_joined_final[sp_joined_final['ncbi_taxon_id'].isna() & sp_joined_final['aphia_id'].isna()]

print('total rows:', sp_joined_final.shape[0])
print('rows missing both ids:', missing_both.shape[0])

total rows: 19816
rows missing both ids: 31


In [ ]:
missing_both.head()

,ncbi_taxon_id,species,subspecies,family_x,target_list_status,sequencing_status,synonym,publication_id,aphia_id,url,...,FBname,DemersPelag,Subfamily,depth_min_in_m,depth_max_in_m,common_depth_min,common_depth_max,length_max_in_cm,common_length_in_cm,weight_max_in_g
961,NaN,Atherinomorus sp.,-,Atherinidae,long_list,data_generation,-,-,NaN,,...,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1083,NaN,Stemonosudis similis,-,Paralepididae,long_list,-,Stemonosudis similis,-,NaN,,...,Indonesian slender barracudina,bathypelagic,Paralepidinae,NaN,70.0,NaN,NaN,3.77,NaN,NaN
1895,NaN,Selene spixii,-,Carangidae,long_list,-,Selene spixii,-,NaN,,...,None,demersal,Caranginae,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2210,NaN,Coris variegata,-,Labridae,long_list,-,Coris variegata,-,NaN,,...,Dapple coris,reef-associated,Corinae,NaN,10.0,NaN,NaN,20.00,NaN,NaN
2215,NaN,Halichoeres bleekeri,-,Labridae,long_list,-,Halichoeres bleekeri,-,NaN,,...,None,demersal,Corinae,NaN,NaN,NaN,NaN,12.20,NaN,NaN


There are 31 records have neither `aphia_id` nor `ncbi_taxon_id`. We move them into a separate csv (`species_missing_both_ids.csv`) for record keeping then drop them from the current working dataset.

In [ ]:
# save missing 31 rows into a separate csv for reference
missing_both.to_csv('C:/Users/30931/OneDrive/Desktop/ocean genome/all fishbase/species_missing_both_ids.csv', index=False)

# remove them from the main dataset
sp_joined_clean = sp_joined_final.drop(missing_both.index)

print('original rows: ', sp_joined_final.shape[0])
print('After removing missing-both rows: ', sp_joined_clean.shape[0])

original rows:  19816
After removing missing-both rows:  19785


Then we're ready for ID consistency checks, synonym collapsing and merging.

### Check ID consistency

In [ ]:
# aphia_id conflicts
aphia_id_consistency = sp_joined_clean.groupby('aphia_id')['species'].nunique()
aphia_conflicts = aphia_id_consistency[aphia_id_consistency > 1]

print("Number of aphia_id conflicts:", aphia_conflicts.shape[0])
print(aphia_conflicts.tail())

Number of aphia_id conflicts: 90
aphia_id
1605280.0    2
1605307.0    2
1605308.0    2
1605309.0    2
1610891.0    2
Name: species, dtype: int64


In [ ]:
# ncbi_taxon_id conflicts
ncbi_id_consistency = sp_joined_clean.groupby('ncbi_taxon_id')['species'].nunique()
ncbi_conflicts = ncbi_id_consistency[ncbi_id_consistency > 1]

print("Number of ncbi_taxon_id conflicts:", ncbi_conflicts.shape[0])
print(ncbi_conflicts.head())

Number of ncbi_taxon_id conflicts: 89
ncbi_taxon_id
8258.0     2
48193.0    2
51789.0    2
52129.0    2
62965.0    2
Name: species, dtype: int64


In [ ]:
aphia_conflict_species = (
    sp_joined_clean[sp_joined_clean["aphia_id"].isin(aphia_conflicts.index)][["aphia_id", "ncbi_taxon_id", "species"]]
    .drop_duplicates()
)

ncbi_conflict_species = (
    sp_joined_clean[sp_joined_clean["ncbi_taxon_id"].isin(ncbi_conflicts.index)][["ncbi_taxon_id", "aphia_id", "species"]]
    .drop_duplicates()
)

In [ ]:
# Build a set of pairs from each
aphia_pairs = set(zip(aphia_conflict_species["aphia_id"], aphia_conflict_species["ncbi_taxon_id"]))
ncbi_pairs  = set(zip(ncbi_conflict_species["aphia_id"], ncbi_conflict_species["ncbi_taxon_id"]))

# Overlap
overlap = aphia_pairs & ncbi_pairs
print("Number of overlapping (aphia_id, ncbi_id) pairs:", len(overlap))

# Differences
only_in_aphia = aphia_pairs - ncbi_pairs
only_in_ncbi  = ncbi_pairs - aphia_pairs

print("Only in aphia_conflict_species:", len(only_in_aphia))
print("Only in ncbi_conflict_species:", len(only_in_ncbi))

Number of overlapping (aphia_id, ncbi_id) pairs: 89
Only in aphia_conflict_species: 2
Only in ncbi_conflict_species: 0


In [ ]:
only_in_aphia

{(1605307.0, nan), (1605307.0, nan)}

Most conflicts are the same (self-detected synonyms with both IDs).

Only in aphia = 2 $\rightarrow{}$ is due to `ncbi_taxon_id` missing (NaN).

### Separate synonyms ($\textbf{self-detected}$) vs. mismatches

In [ ]:
# same pair, multiple names (safe synonyms)
synonym_conflicts = (
    sp_joined_clean.groupby(["aphia_id","ncbi_taxon_id"])["species"].nunique().reset_index()
)
synonym_conflicts = synonym_conflicts[synonym_conflicts["species"] > 1]

print("Synonym conflict pairs:", synonym_conflicts.shape[0])

Synonym conflict pairs: 89


In [ ]:
# cross mismatches
aphia_mismatches = aphia_conflict_species.groupby("aphia_id")["ncbi_taxon_id"].nunique()
aphia_mismatches = aphia_mismatches[aphia_mismatches > 1]

ncbi_mismatches = ncbi_conflict_species.groupby("ncbi_taxon_id")["aphia_id"].nunique()
ncbi_mismatches = ncbi_mismatches[ncbi_mismatches > 1]

print("Aphia mismatches:", aphia_mismatches.shape[0])
print("NCBI mismatches:", ncbi_mismatches.shape[0])

Aphia mismatches: 0
NCBI mismatches: 0


In [ ]:
len(sp_joined_clean)

19785

We had 0 mismatches, so all safe.

#### Synonym groups

In [ ]:
# build a flexible key (no new column in the df)
keys = sp_joined_clean.apply(
    lambda r: (
        (r["aphia_id"] if pd.notna(r["aphia_id"]) else pd.NA),
        (r["ncbi_taxon_id"] if pd.notna(r["ncbi_taxon_id"]) else pd.NA)
    ),
    axis=1
)

In [ ]:
synonym_counts = (
    sp_joined_clean
    .groupby(keys)["species"]
    .nunique()
    .reset_index(name="n_species_names")
)

print(synonym_counts["n_species_names"].value_counts().head())

n_species_names
1    19604
2       89
3        1
Name: count, dtype: int64


By self detection, 19604 species has no synonyms, 89 species with 2 names, and 1 species with 3 names.

In [ ]:
# split the tuple key iunto two columns
synonym_counts[["aphia_id","ncbi_taxon_id"]] = pd.DataFrame(
    synonym_counts["index"].tolist(), index=synonym_counts.index
)
synonym_counts = synonym_counts.drop(columns=["index"])

# quick look at the distribution
print(synonym_counts["n_species_names"].value_counts().head())

n_species_names
1    19604
2       89
3        1
Name: count, dtype: int64


### Define attributes to track species with synonyms

We identified 90 species with self-detected synonyms. For these, we should check whether the other attributes like depth, weight and so on are missing for the canonical name but present for one of the synonyms.

In [ ]:
attributes = [
    "depth_min_in_m","depth_max_in_m","common_depth_min","common_depth_max",
    "length_max_in_cm","common_length_in_cm","weight_max_in_g"
]

$\textcolor{blue}{\textbf{Conflict log}}$

In [ ]:
def check_conflicts(df):
  conflicts = {}
  for attr in attributes:
    vals = df[attr].dropna().unique()
    if len(vals) > 1:
      conflicts[attr] = vals.tolist()
  return conflicts if conflicts else None

# Only synonym groups (IDs of group that actually have synonyms >1 names)
synonym_ids = synonym_counts.loc[synonym_counts["n_species_names"] > 1, ["aphia_id","ncbi_taxon_id"]]

# Add species names to the conflict log
conflict_log_all = (
    sp_joined_clean.groupby(keys)
    .apply(check_conflicts)
    .dropna()
    .reset_index()
    .rename(columns={"index":"_key", 0:"conflicting_attributes"})
)

# split tuple key to two columns (no group_key stored in our df)
conflict_log_all[["aphia_id","ncbi_taxon_id"]] = pd.DataFrame(
    conflict_log_all["_key"].tolist(), index=conflict_log_all.index
)
conflict_log_all = conflict_log_all.drop(columns=["_key"])

# Ensure both have the same dtypes
for col in ["aphia_id","ncbi_taxon_id"]:
    conflict_log_all[col] = conflict_log_all[col].astype("Int64")
    synonym_ids[col] = synonym_ids[col].astype("Int64")

# keep only the synonym groups in the conflict log
conflict_log = conflict_log_all.merge(
    synonym_ids, on=["aphia_id","ncbi_taxon_id"], how="inner"
)

In [ ]:
# add species names as a list per group
species_names = (
    sp_joined_clean
    .groupby(keys)["species"]
    .apply(lambda x: sorted(x.unique()))
    .reset_index()
    .rename(columns={"index":"_key", "species":"species_names"})
)

species_names[["aphia_id","ncbi_taxon_id"]] = pd.DataFrame(
    species_names["_key"].tolist(), index=species_names.index
)
species_names = species_names.drop(columns=["_key"])

# align dtypes across all ID columns
for col in ["aphia_id", "ncbi_taxon_id"]:
    conflict_log[col]   = conflict_log[col].astype("Int64")
    species_names[col]  = species_names[col].astype("Int64")

# merge species names into conflict_log (two_id columns preserved)
conflict_log = conflict_log.merge(
    species_names[["aphia_id","ncbi_taxon_id","species_names"]],
    on=["aphia_id","ncbi_taxon_id"],
    how="left")

conflict_log.head(10)

,conflicting_attributes,aphia_id,ncbi_taxon_id,species_names
0,"{'length_max_in_cm': [100.0, 40.0]}",126983,48193,"[Mugil cephalus, Mugil galapagensis]"
1,"{'depth_max_in_m': [70.0, 150.0], 'length_max_...",127008,349657,"[Pseudotolithus senegalensis, Pseudotolithus s..."
2,"{'depth_min_in_m': [94.0, 39.0], 'depth_max_in...",274100,215424,"[Lycodes gracilis, Lycodes vahlii]"
3,"{'depth_min_in_m': [18.0, 3.0], 'depth_max_in_...",275760,288138,"[Halichoeres cyanocephalus, Halichoeres dimidi..."
4,"{'depth_max_in_m': [24.0, 30.0], 'length_max_i...",275772,51789,"[Halichoeres maculipinna, Halichoeres penrosei]"
5,"{'length_max_in_cm': [23.0, 15.899999618530273...",278412,433688,"[Takifugu alboplumbeus, Takifugu niphobles]"
6,"{'length_max_in_cm': [7.0, 12.0]}",278919,1609328,"[Paramonacanthus cryptodon, Paramonacanthus ja..."
7,"{'depth_min_in_m': [80.0, 100.0]}",280414,62965,"[Crassinarke dormitor, Narke japonica]"
8,"{'depth_min_in_m': [200.0, 563.0], 'depth_max_...",1007248,1932029,"[Eptatretus fernholmi, Eptatretus luzonicus]"


We found 9 groups with conflicting features $\rightarrow{}$ we save for manual review.

In [ ]:
# export conflicts cases into csv
conflict_log.to_csv("C:/Users/30931/OneDrive/Desktop/ocean genome/all fishbase/conflict_log.csv", index=False)

Only 9 groups have true attribute conflicts ($\textcolor{purple}{\text{listed above}}$).

### Collapse synonyms into master table

In [ ]:
def collapse_group(df):
  """
  Collapse a group of rows (same aphia_id, ncbi_taxon_id) into one row.

  Parameters
  ----------
  df : pd.DataFrame
     Subset of the dataframe for a single (aphia_id, ncbi_taxon_id) group.
  attributes : list
     List of attribute column names to check for consistency.

  Returns
  -------
  dict
      A dictionary representing the collapsed row with:
      - aphia_id
      - ncbi_taxon_id
      - species_canonical
      - synonyms
      - attributes (filled, or NA if conflicting/absent)
  """
  names = sorted(df["species"].dropna().unique())
  canonical = names[0] if len(names) else pd.NA
  synonyms = names[1:] if len(names) > 1 else []

  # take which ever ID(s) exist in the group
  aphia_val = df['aphia_id'].dropna()
  ncbi_val = df['ncbi_taxon_id'].dropna()

  row = {"aphia_id": df["aphia_id"].iloc[0] if not aphia_val.empty else pd.NA,
         "ncbi_taxon_id": df["ncbi_taxon_id"].iloc[0] if not ncbi_val.empty else pd.NA,
         "species_canonical": canonical,
         "synonyms_id": synonyms}

  # For each attribute: if consistent → keep; if conflicting → set NA
  for attr in attributes:
    # Check if attribute column exists before accessing it
    if attr in df.columns:
      vals = df[["species", attr]].dropna(subset=[attr])
      uniq = vals[attr].unique()
      if len(uniq) == 0:
        row[attr] = pd.NA
      elif len(uniq) == 1:
        row[attr] = uniq[0]   # completely missing
      else:
        # multiplt values; if canonical missing take a synonym value, else mark NA
        canon_vals = vals.loc[vals["species"] == canonical, attr]
        row[attr] = (uniq[0] if canon_vals.empty or canon_vals.isna().all() else pd.NA)
    else:
      row[attr] = pd.NA # Attribute column not found

  return row

### Split safe (non-conflicts) vs. conflicted

In [ ]:
# Apply collapse
master_species = (
    sp_joined_clean
    .groupby(keys)
    .apply(collapse_group)
    .apply(pd.Series)
    .reset_index(drop=True)
)

# make ID dtypes consistent & nullable
for col in ["aphia_id","ncbi_taxon_id"]:
    master_species[col] = master_species[col].astype("Int64")
    conflict_log[col] = conflict_log[col].astype("Int64")

# build a set of keys for conflicted groups
conflict_keys = set(map(tuple, conflict_log[["aphia_id","ncbi_taxon_id"]].to_numpy()))

# keeps all groups EXCEPT those 9 conflicted records
master_safe = master_species[
    ~master_species[["aphia_id","ncbi_taxon_id"]].apply(tuple, axis=1).isin(conflict_keys)
].copy()

#
master_conflicted = master_species[
    master_species[["aphia_id","ncbi_taxon_id"]].apply(tuple, axis=1).isin(conflict_keys)
].copy()

master_safe.to_csv("C:/Users/30931/OneDrive/Desktop/ocean genome/all fishbase/master_species_clean.csv", index=False)
master_conflicted.to_csv("C:/Users/30931/OneDrive/Desktop/ocean genome/all fishbase/master_species_conflicts_rows.csv", index=False)

print("Master safe:", master_safe.shape)
print("Master conflicted:", master_conflicted.shape)

Master safe: (19685, 11)
Master conflicted: (9, 11)


In [ ]:
master_conflicted.head()

,aphia_id,ncbi_taxon_id,species_canonical,synonyms_id,depth_min_in_m,depth_max_in_m,common_depth_min,common_depth_max,length_max_in_cm,common_length_in_cm,weight_max_in_g
772,126983,48193,Mugil cephalus,[Mugil galapagensis],0.0,120.0,0.0,10.0,<NA>,50.0,<NA>
795,127008,349657,Pseudotolithus senegalensis,[Pseudotolithus senegallus],0.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
7213,274100,215424,Lycodes gracilis,[Lycodes vahlii],<NA>,<NA>,71.0,<NA>,<NA>,27.5,<NA>
8733,275760,288138,Halichoeres cyanocephalus,[Halichoeres dimidiatus],<NA>,<NA>,30.0,<NA>,<NA>,<NA>,<NA>
8744,275772,51789,Halichoeres maculipinna,[Halichoeres penrosei],2.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


- `master_safe`: rows with consistent trait values, good for merging and doing analysis;
- `master_conflicted`: rows where synonyms disagree on traits, used for manual review.

### Merge in original `synonym` column (using `master_safe`)

In [ ]:
# bring original synonym from the raw dataset
orig_syn = sp_joined_clean[["aphia_id","ncbi_taxon_id","synonym"]].drop_duplicates()

# merge into the safe table
master_safe = master_safe.merge(orig_syn, on=["aphia_id","ncbi_taxon_id"], how="left")

# combine synonym_id + original text synonyms
def combine_syn(row):
  lst = row['synonyms_id'] if isinstance(row['synonyms_id'], list) else []
  txt = [] if pd.isna(row.get('synonym')) else [s.strip() for s in str(row["synonym"]).split(";") if s.strip()]
  return set(lst+txt)

In [ ]:
master_safe['all_synonyms'] = master_safe.apply(combine_syn, axis=1)

# drop the old raw synonym column (no need anymore)
master_safe1 = master_safe.drop(columns=['synonym', 'synonyms_id'])

In [ ]:
master_safe1.head()

,aphia_id,ncbi_taxon_id,species_canonical,depth_min_in_m,depth_max_in_m,common_depth_min,common_depth_max,length_max_in_cm,common_length_in_cm,weight_max_in_g,all_synonyms
0,101170,7769,Myxine glutinosa,20.0,1200.0,27.0,460.0,95.0,30.0,<NA>,{-}
1,101171,<NA>,Myxine ios,614.0,1625.0,<NA>,<NA>,57.0,<NA>,<NA>,{Myxine ios}
2,101172,7748,Lampetra fluviatilis,10.0,<NA>,<NA>,<NA>,50.0,35.0,150.0,{-}
3,101173,980415,Lethenteron camtschaticum,0.0,50.0,<NA>,<NA>,63.0,16.049999,200.0,{-}
4,101174,7757,Petromyzon marinus,1.0,4099.0,<NA>,<NA>,120.0,60.0,2500.0,{-}


### Export the final `master_safe`

In [ ]:
print(master_safe1['all_synonyms'].head())
print(master_safe1['all_synonyms'].apply(type).value_counts().head())

0             {-}
1    {Myxine ios}
2             {-}
3             {-}
4             {-}
Name: all_synonyms, dtype: object
all_synonyms
<class 'set'>    19686
Name: count, dtype: int64


In [ ]:
# turn list into semicolon-separated string for csv
def to_semicolon(v):
  if isinstance(v, (list, tuple, set)):
    return '; '.join(sorted({str(x) for x in v if pd.notna(x)}))
  if pd.isna(v):
    return ''
  return str(v)

master_safe1['all_synonyms'] = master_safe1['all_synonyms'].apply(to_semicolon)

In [ ]:
# columns to keep from original clean data
meta_cols = [
    "aphia_id","ncbi_taxon_id",
    "subspecies","target_list_status","sequencing_status","status","publication_id","rank",
    "isMarine","isBrackish","isFreshwater","isTerrestrial","isExtinct","modified","spec_code",
    "order","family_x","Genus","authority","FBname","DemersPelag","Subfamily"
]

meta_cols = [c for c in meta_cols if c in sp_joined_clean.columns]

# deduplicate metadata by ID pair
fb1_meta = (
    sp_joined_clean[meta_cols]
    .drop_duplicates(subset=["aphia_id","ncbi_taxon_id"])
    .copy()
)

# align ID dtypes to avoid merge errors
for col in ["aphia_id","ncbi_taxon_id"]:
    fb1_meta[col]    = pd.to_numeric(fb1_meta[col], errors="coerce").astype("Int64")
    master_safe1[col]= pd.to_numeric(master_safe1[col], errors="coerce").astype("Int64")

# Left-merge metadata into your safe table (traits stay from master_safe1)
master_final = master_safe1.merge(fb1_meta, on=["aphia_id","ncbi_taxon_id"], how="left")

trait_cols = [c for c in [
    "depth_min_in_m","depth_max_in_m",
    "common_depth_min","common_depth_max",
    "length_max_in_cm","common_length_in_cm","weight_max_in_g"
] if c in master_final.columns]

ordered = (
    ["aphia_id","ncbi_taxon_id","species_canonical","all_synonyms"] +
    [c for c in ["publication_id", "family_x",
                 "subspecies",'authority', "modified", "status","rank","order","target_list_status","sequencing_status",
                 "isMarine","isBrackish","isFreshwater","isTerrestrial","isExtinct",
                 "spec_code","Genus", "FBname","DemersPelag","Subfamily"]
     if c in master_final.columns] +
    trait_cols
)

master_final = master_final[[c for c in ordered if c in master_final.columns]]

In [ ]:
master_final.head()

,aphia_id,ncbi_taxon_id,species_canonical,all_synonyms,publication_id,family_x,subspecies,authority,modified,status,...,FBname,DemersPelag,Subfamily,depth_min_in_m,depth_max_in_m,common_depth_min,common_depth_max,length_max_in_cm,common_length_in_cm,weight_max_in_g
0,101170,7769,Myxine glutinosa,-,-,Myxinidae,-,"Linnaeus, 1758",2008-01-15T17:27:08.177Z,accepted,...,Atlantic hagfish,bathydemersal,Myxininae,20.0,1200.0,27.0,460.0,95.0,30.0,<NA>
1,101171,<NA>,Myxine ios,Myxine ios,-,Myxinidae,-,"Fernholm, 1981",2008-01-15T17:27:08.177Z,accepted,...,White-headed hagfish,bathydemersal,Myxininae,614.0,1625.0,<NA>,<NA>,57.0,<NA>,<NA>
2,101172,7748,Lampetra fluviatilis,-,-,Petromyzontidae,-,"(Linnaeus, 1758)",2017-08-09T06:21:34.107Z,accepted,...,River lamprey,demersal,Lampetrinae,10.0,<NA>,<NA>,<NA>,50.0,35.0,150.0
3,101173,980415,Lethenteron camtschaticum,-,-,Petromyzontidae,-,"(Tilesius, 1811)",2010-05-06T09:57:44.313Z,accepted,...,Arctic lamprey,demersal,Lampetrinae,0.0,50.0,<NA>,<NA>,63.0,16.049999,200.0
4,101174,7757,Petromyzon marinus,-,-,Petromyzontidae,-,"Linnaeus, 1758",2008-01-15T17:27:08.177Z,accepted,...,Sea lamprey,demersal,Petromyzontinae,1.0,4099.0,<NA>,<NA>,120.0,60.0,2500.0


In [ ]:
#export
master_final.to_csv('C:/Users/30931/OneDrive/Desktop/ocean genome/all fishbase/non_conflicted_species_final.csv', index = False)
print('final shape: ', master_final.shape)

final shape:  (19686, 31)


In [ ]:
master_final.to_csv("master_final.csv", index=False)

Add temperature features from Fishbase

In [ ]:
import rpy2.robjects as ro
from rpy2.robjects import conversion, pandas2ri
import pandas as pd

# New conversion method (replaces pandas2ri.activate())
converter = conversion.Converter('pandas conversion', template=conversion.converter)
converter += pandas2ri.converter

with converter.context():
    # Loading R packages
    ro.r('library(tidyverse)')
    ro.r('library(rfishbase)')
    ro.r('library(stringr)')
    ro.r('library(dplyr)')

    # === Step 1. Read in your original CSV ===
    ro.r('master_final <- read.csv("master_final.csv")')

    # Take unique species_canonical + validate_names
    ro.r('my_species2 <- unique(master_final$species_canonical)')
    ro.r('valid_names2 <- validate_names(my_species2)')

    # === Step 2. Pull stocks data (only a few fields) ===
    ro.r('st2 <- stocks(species_list = valid_names2, fields = c("Species","TempMin","TempMax"))')

    # === Step 3. Clean the data (select the row with the least missing TempMin/TempMax values) ===
    ro.r('''
    st2_clean <- st2 %>%
      mutate(
        .rowid   = row_number(),
        na_count = rowSums(across(c(TempMin, TempMax), ~ is.na(.x)))
      ) %>%
      group_by(Species) %>%
      arrange(na_count, .rowid, .by_group = TRUE) %>%
      slice(1) %>%
      ungroup() %>%
      select(-na_count, -.rowid)
    ''')

    # === Step 4. Merge back to the original table ===
    ro.r('''
    final_species_python <- master_final %>%
      left_join(st2_clean, by = c("species_canonical" = "Species"))
    ''')

    # === Step 5. Convert to a pandas DataFrame, Python-compatible ===
    final_species_python = ro.r('final_species_python')
    final_species_python = pd.DataFrame(final_species_python)

print(final_species_python.head())

Joining with `by = join_by(SpecCode)`
   aphia_id  ncbi_taxon_id          species_canonical all_synonyms  \
1    101170           7769           Myxine glutinosa            -   
2    101171    -2147483648                 Myxine ios   Myxine ios   
3    101172           7748       Lampetra fluviatilis            -   
4    101173         980415  Lethenteron camtschaticum            -   
5    101174           7757         Petromyzon marinus            -   

  publication_id         family_x subspecies         authority  \
1              -        Myxinidae          -    Linnaeus, 1758   
2              -        Myxinidae          -    Fernholm, 1981   
3              -  Petromyzontidae          -  (Linnaeus, 1758)   
4              -  Petromyzontidae          -  (Tilesius, 1811)   
5              -  Petromyzontidae          -    Linnaeus, 1758   

                   modified    status  ...        Subfamily depth_min_in_m  \
1  2008-01-15T17:27:08.177Z  accepted  ...        Myxininae       

In [ ]:
import pandas as pd

SENTINEL = -2147483648
final_species_python["ncbi_taxon_id"] = (
    final_species_python["ncbi_taxon_id"]
    .replace(SENTINEL, pd.NA)
    .astype("Int64")
)


In [ ]:
import pandas as pd

SENTINEL = -2147483648
final_species_python["aphia_id"] = (
    final_species_python["aphia_id"]
    .replace(SENTINEL, pd.NA)
    .astype("Int64")
)

In [ ]:

# save to csv
final_species_python.to_csv("final_species_python.csv", index=False)

In [ ]:
import pandas as pd

df = pd.read_csv("final_species_python.csv")

print("rows:", df.shape[0])
print("columns:", df.shape[1])

rows: 19686
columns: 33
